In [ ]:
%matplotlib inline

# Polynomial chaos expansion (PCE).

A [PCERegressor][gemseo.mlearning.regression.algos.pce.PCERegressor] is a PCE model
based on [OpenTURNS](http://openturns.github.io/).


In [ ]:
from __future__ import annotations

from matplotlib import pyplot as plt
from numpy import array

from gemseo import create_discipline
from gemseo import create_parameter_space
from gemseo import sample_disciplines
from gemseo.mlearning import create_regression_model

## Problem

In this example,
we represent the function $f(x)=(6x-2)^2\sin(12x-4)$
by the [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline].

!!! quote "References"
      Alexander I. J. Forrester, Andras Sobester, and Andy J. Keane.
      Engineering design via surrogate modelling: a practical guide. Wiley, 2008.



In [ ]:
discipline = create_discipline(
    "AnalyticDiscipline",
    name="f",
    expressions={"y": "(6*x-2)**2*sin(12*x-4)"},
)

and seek to approximate it over the input space



In [ ]:
input_space = create_parameter_space()
input_space.add_random_variable("x", "OTUniformDistribution")

To do this,
we create a training dataset with 6 equispaced points:



In [ ]:
training_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="PYDOE_FULLFACT", n_samples=10
)

## Basics

### Training

Then,
we train an PCE regression model from these samples:



In [ ]:
model = create_regression_model("PCERegressor", training_dataset)
model.learn()

### Prediction

Once it is built,
we can predict the output value of $f$ at a new input point:



In [ ]:
input_value = {"x": array([0.65])}
output_value = model.predict(input_value)
output_value

as well as its Jacobian value:



In [ ]:
jacobian_value = model.predict_jacobian(input_value)
jacobian_value

### Plotting

Of course,
you can see that the quadratic model is no good at all here:



In [ ]:
test_dataset = sample_disciplines(
    [discipline], input_space, "y", algo_name="PYDOE_FULLFACT", n_samples=100
)
input_data = test_dataset.get_view(variable_names=model.input_names).to_numpy()
reference_output_data = test_dataset.get_view(variable_names="y").to_numpy().ravel()
predicted_output_data = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.grid()
plt.legend()
plt.show()

## Settings

The [PCERegressor][gemseo.mlearning.regression.algos.pce.PCERegressor] has many options
defined in the [PCERegressor_Settings][gemseo.mlearning.regression.algos.pce_settings.PCERegressor_Settings] Pydantic model.

### Degree




In [ ]:
model = create_regression_model("PCERegressor", training_dataset, degree=3)
model.learn()

and see that this model seems to be better:



In [ ]:
predicted_output_data_ = model.predict(input_data).ravel()
plt.plot(input_data.ravel(), reference_output_data, label="Reference")
plt.plot(input_data.ravel(), predicted_output_data, label="Regression - Basics")
plt.plot(input_data.ravel(), predicted_output_data_, label="Regression - Degree(3)")
plt.grid()
plt.legend()
plt.show()